# Tools definition

In [1]:
# Define the calculator tool

from langchain.chains import LLMMathChain
from langchain_openai import ChatOpenAI
from langchain.tools import tool


llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125"
)
llm_math = LLMMathChain(llm=llm, verbose=True)

@tool
def calculator(query: str) -> str:
    """This is the LLMath tool, use when a mathematical operation is required"""
    return llm_math.run(query)

/langchain_env/lib/python3.12/site-packages/langchain/chains/llm_math/base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [2]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="This is the super powerfull mathematical python REPL tool",
    func=python_repl.run,
)

# Model creation

In [3]:
# create the model

from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function

model = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125"
).bind(functions=[convert_to_openai_function(calculator),
                  convert_to_openai_function(repl_tool)])


# Chat agent building

In [4]:
from langchain.prompts import ChatMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

input_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot."),
        ("human", "{user_input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad", optional=True)
    ]
)


In [5]:
from langchain.schema.output_parser import StrOutputParser

output_parser = StrOutputParser()

In [6]:
from langchain_core.runnables import RunnablePassthrough
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

runnable_agent_scratchpad = RunnablePassthrough.assign(
    agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
)


chain = runnable_agent_scratchpad | input_prompt | model | OpenAIFunctionsAgentOutputParser()

In [7]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.agent import AgentFinish


def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = chain.invoke({
            "user_input": user_input, 
            "intermediate_steps": intermediate_steps
        })
        if isinstance(result, AgentFinish):
            return result.return_values['output']
        tool_obtained = {
             "calculator": calculator,
             "python_repl": python_repl,
        }[result.tool]
        observation = tool_obtained.run(result.tool_input)
        intermediate_steps.append((result, observation))


# Demo prompts

In [8]:
# example of agent execution

# reponse = run_agent("What are the tools available?")
# print(reponse)

In [9]:
# reponse = run_agent("What is the result of 2+2?")
# print(reponse)

In [10]:
# reponse = run_agent("""Use the python_repl tool generate a random matrix of size 5x5 and return the result in the following format:
#                        [[1, 2],
#                         [3, 4]]                
#                     """)
# print(reponse)

In [11]:
# reponse = run_agent("""Use the python_repl tool generate a random square matrix of size 50000 and calculate the inverse matrix.Please return the result in the following format:
#                        [[1, 2],
#                         [3, 4]]                
#                     """)
# print(reponse)

# Chatbot Demo

In [12]:
import panel as pn
import param

class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])

    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.agent_execution = run_agent

    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        result = self.agent_execution(user_input=query)
        self.chat_history.extend([(query, result)])
        self.answer = result
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=600)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600))
        ])
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)

    @param.depends('convchain', 'clr_history') 
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist=[pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F6'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    def clr_history(self,count=0):
        self.chat_history = []
        return 


In [13]:
pn.extension()

In [14]:
cb = cbfs()

button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
button_clearhistory.on_click(cb.clr_history)
inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp) 


tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=300),
    pn.layout.Divider(),
)

tab3= pn.Column(
    pn.panel(cb.get_chats),
    pn.layout.Divider(),
)

tab4=pn.Column(
    pn.Row( button_clearhistory, pn.pane.Markdown("Clears chat history. Can use to start a new topic" ))
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# Chat Bot demo DeepLearning AI')),
    pn.Tabs(('Conversation', tab1), ('Chat History', tab3),('Configure', tab4))
)


dashboard

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=WidgetBox, defer_load=False, height=300, loading_indicator=True)
            [3] Divider()
        [1] Column
            [0] ParamMethod(method, _pane=WidgetBox, defer_load=False)
            [1] Divider()
        [2] Column
            [0] Row
                [0] Button(button_type='warning', name='Clear History')
                [1] Markdown(str)